## JSON Homework
Realizado por: Sergio Duque Mejía. C.C: 1036654435

In [214]:
import json
import requests
import pandas as pd
import numpy as np
import wget
import re

In [215]:
# Descarga de datos de paises
url = 'https://pkgstore.datahub.io/core/country-list/data_json/data/8c458f2d15d9f2119654b29ede6e45b8/data_json.json'
filename = wget.download(url)

### Seleccion de pais de forma aleatoria

In [227]:
data = open('data_json.json')
country_json = json.load(data)
rand_index = np.random.randint(len(country_json)) # numero aleatorio en el rango de los elementos dentro del JSON
country_name = country_json[rand_index]['Name'] # Selecciona elemento dentro del JSON aleatorio
print(country_name)

Georgia


### Busca las instituciones del pais seleccionado

In [228]:
url2 = "https://inspirehep.net/api/institutions?q={}".format(country_name)
institution_link = requests.get(url2)

In [229]:
legacy_ICN = []
for institution in institution_link.json().get('hits').get('hits'):
    n_papers = institution.get('metadata').get('number_of_papers')
    if n_papers > 0:
        icn = institution.get('metadata').get('legacy_ICN') # Toma las instituciones que cumple la condicion
        legacy_ICN.append(icn)

print(legacy_ICN) # instituciones en pais seleccionado
print(len(legacy_ICN)) # numero de elementos obtenidos

['Unlisted, GE', 'U. Georgia, Athens', 'Georgia Tech., Atlanta', 'West Georgia U.', 'Georgia Tech', 'Sukhumi, FTI', 'Agnes Scott Coll.', 'Covenant Coll.', 'Lockheed, Marietta', 'General Analytics, Athens']
10


### Imprime Nombre de autor, Titulo de Articulo y numero de referencia del autor

In [230]:
ref = []

for ICN in legacy_ICN: # Selecciona todas las instituciones encontradas anteriormente
    url3 = "https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{}+and+ac+1->+10".format(ICN)
    literature_link = requests.get(url3)
    for literature in literature_link.json().get('hits').get('hits'):
        author_temp = literature.get('metadata').get('authors')[0] # Selecciona los autores
        print(author_temp['full_name_unicode_normalized']) # Nombre completo del autor
        print(literature.get('metadata').get('titles')[0]['title']) # Selecciona el nombre del articulo
        try:
            link = str(author_temp['record']['$ref']) # Selecciona el link que redirige al autor, se usa try por que a veces no hay link
            ref_temp = "".join(re.findall("\d",link)) # Obtenemos el numero de referencia del autor
            print('referencia web author: '+ref_temp+'\n')
            ref.append(ref_temp) # Almacena el numero de referencia de los autores que tienen link solamente
        except:
            print("no referencia web author\n") # NO almacena ningun autor que no tenga link

khvedelidze, arsen
Generalizing Stratonovich-Weyl axioms for composite systems
referencia web author: 1096932

kahlhoefer, felix
An introduction to DDCalc
referencia web author: 1078191

abgaryan, vahagn
Kenfack Zyczkowski indicator of nonclassicality for two non-equivalent representations of Wigner function of qutrit
referencia web author: 1939254

khunjua, t.g.
Electrical neutrality and $\beta$-equilibrium conditions in dense quark matter: generation of charged pion condensation by chiral imbalance
referencia web author: 1078163

sanz, c.
A rotary mount for positioning a stretched wire axis within a coordinate measurement machine
referencia web author: 1673639

heinze, martin
Isometry Group Orbit Quantization of Spinning Strings in AdS$_{3} \times$ S$^3$
referencia web author: 1072559

sharmazanashvili, a.
A Tool for Conditions Tag Management in ATLAS
referencia web author: 1911904

bechtle, philip
$\mathsf{HiggsBounds}-4$: Improved Tests of Extended Higgs Sectors against Exclusion B

### Almacena nombre, email, rango, institucion de cada autor con link de referencia

Se usa try y except para cada valor debido a que puede tener al menos uno, se completa el resultado con un None indicando que no tiene esa informacion

In [276]:
def author_info(req):
    aut = req.json()['metadata'] # datos de cada autor
    try: email = aut['email_addresses'][0]['value'] # email
    except: email = 'None'
    try: name = aut['name']['value'] # nombre
    except: name = 'None'
    try: rank = aut['positions'][0]['rank'] # rango
    except: rank = 'None'
    try: inst = aut['positions'][0]['institution'] # institucion
    except: inst = 'None'
    return name,email,rank,inst

In [278]:
name = []
email = []
rank = []
inst = []

for reference in ref:
    url4 = "https://inspirehep.net/api/authors/{}".format(reference)
    author_link = requests.get(url4)
    n_temp,e_temp,r_temp,i_temp = author_info(author_link) # obtiene nombre,email,rango e institucion
    name.append(n_temp)
    email.append(e_temp)
    rank.append(r_temp)
    inst.append(i_temp)

In [279]:
table = {'name':name,'email':email,'rank':rank,'institution':inst}
dataframe = pd.DataFrame(table) # crea dataframe basado en la obtencion de datos
dataframe.head()

,name,email,rank,institution
0,"Khvedelidze, Arsen M.",akhved@jinr.ru,SENIOR,"GTU,Tbilisi"
1,"Kahlhoefer, Felix",kahlhoefer@physik.rwth-aachen.de,None,RWTH Aachen U.
2,"Abgaryan, V.",None,None,None
3,"Khunjua, Tamaz G.",t.khunjua@ug.edu.ge,SENIOR,"Georgia U., Tbilisi"
4,"Sanz, Claude",None,None,CERN


puede existir autores repetidos dado que se referencio mas de una veces debido a que para una institucion especifica puede tener mas de un articulo, por lo que se elimina los duplicados.

In [290]:
dataframe.drop_duplicates(subset='name',inplace=True) # 

In [292]:
dataframe

,name,email,rank,institution
0,"Khvedelidze, Arsen M.",akhved@jinr.ru,SENIOR,"GTU,Tbilisi"
1,"Kahlhoefer, Felix",kahlhoefer@physik.rwth-aachen.de,None,RWTH Aachen U.
2,"Abgaryan, V.",None,None,None
3,"Khunjua, Tamaz G.",t.khunjua@ug.edu.ge,SENIOR,"Georgia U., Tbilisi"
4,"Sanz, Claude",None,None,CERN
5,"Heinze, Martin Bernhard Johannes",martin.heinze@desy.de,POSTDOC,"U. Hamburg, Inst. Theor. Phys. II"
6,"Sharmazanashvili, Alexander",None,None,None
7,"Bechtle, Philip",philip.bechtle@desy.de,None,DESY
8,"Jorjadze, George",None,None,Razmadze Math. Inst.
10,"Geller, Michael",None,JUNIOR,Tel Aviv U. (main)
